In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage
import math
import io

In [16]:
#Get google credentials downloaded by terraform
file_path = "../terraform/gcp_credentials.json"
credentials = service_account.Credentials.from_service_account_file(file_path)

In [38]:
#Add your project name here 
client = bigquery.Client(credentials=credentials, project="dte-de-course-454810")
storage_client = storage.Client()

In [ ]:
# Should be - bigquery-public-data, ncaa_basketball
project_id = "bigquery-public-data"
dataset_id = "ncaa_basketball"

In [19]:
dataset_ref = client.dataset(dataset_id, project=project_id)
tables = list(client.list_tables(dataset_ref))

In [ ]:
# Should be - ncaa-bucket-dump
bucket_name = "kestra-storage-1997"
bucket = storage_client.get_bucket(bucket_name)

In [42]:
##for table in tables:

table_name = tables[1].table_id

query_job = client.query(f"""
                             SELECT *
                             FROM `{project_id}.{dataset_id}.{table_name}`
                             """)
df = query_job.to_dataframe()

chunk_size = 10000  
num_chunks = math.ceil(len(df) / chunk_size)

for i in range(num_chunks):
    df_chunk = df.iloc[i*chunk_size:(i+1)*chunk_size]

    csv_buffer = io.BytesIO()

    # Write to Buffer
    compression_opts = dict(method="zip", archive_name=f"{table_name}_part_{i+1}.csv")
    df_chunk.to_csv(csv_buffer, index=True, compression=compression_opts)

    # Go to the start of thebuffer and load to bucket
    csv_buffer.seek(0)  
    blob = bucket.blob(f"{table_name}/{table_name}_part_{i+1}.zip")
    blob.upload_from_file(csv_buffer)